In [79]:
import kfp
import kfp.components as components
import kfp.dsl as dsl
from kfp.components import InputPath, OutputPath

In [80]:
def train_export_model(trainingjobName: str, epochs: str, version: str):
    
    import tensorflow as tf
    from numpy import array
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense
    from tensorflow.keras.layers import Flatten, Dropout, Activation
    from tensorflow.keras.layers import LSTM
    import numpy as np
    print("numpy version")
    print(np.__version__)
    import pandas as pd
    import os
    from featurestoresdk.feature_store_sdk import FeatureStoreSdk
    from modelmetricsdk.model_metrics_sdk import ModelMetricsSdk
    
    fs_sdk = FeatureStoreSdk()
    mm_sdk = ModelMetricsSdk()
    
    features = fs_sdk.get_features(trainingjobName, ['DRB.UEThpDl','RRU.PrbUsedDl','RRC.ConnMean','QosFlow.TotPdcpPduVolumeDl', 'PEE.AvgPower', 'timestamp'])
    
    print("all feature:")
    print(features.columns)
    
    name_file="https://drive.usercontent.google.com/download?id=1DoB_MJHXDRaKvjAwHzPNZHdmx0Gl87nd&export=download&authuser=0&confirm=t&uuid=798deac7-ba99-4116-bb8b-7cb3ef043ce8&at=APZUnTUg8mVcxXbOXkJx3qS3K9cu:1723001283357"
    import urllib.request
    urllib.request.urlretrieve(
            name_file, 'model_tensor2.hdf5')
    
    print("url succeeded")
    
    model = tf.keras.models.load_model('./model_tensor2.hdf5')
    
    print("load mymodel succeeded ")
    print(model.summary())
    model.save("./")
    xx=[0]
    yy=[0]
    import json
    data = {}
    data['metrics'] = []
    data['metrics'].append({'Accuracy': str(np.mean(np.absolute(np.asarray(xx)-np.asarray(yy))<5))})
    
    mm_sdk.upload_metrics(data, trainingjobName, version)
    mm_sdk.upload_model("./", trainingjobName, version)

In [81]:
BASE_IMAGE = "traininghost/pipelineimage:latest"

In [82]:
def train_and_export(trainingjobName: str, epochs: str, version: str):
    trainOp = components.func_to_container_op(train_export_model, base_image=BASE_IMAGE)(trainingjobName, epochs,version)
    # Below line to disable caching of pipeline step
    trainOp.execution_options.caching_strategy.max_cache_staleness = "P0D"
    trainOp.container.set_image_pull_policy("IfNotPresent")

In [83]:
@dsl.pipeline(
    name="qoetest",
    description="qoe",
)
def super_model_pipeline( 
    trainingjob_name: str, epochs: str, version: str):
    
    train_and_export(trainingjob_name, epochs, version)

In [84]:
pipeline_func = super_model_pipeline
file_name = "qoe_model_pipeline"

kfp.compiler.Compiler().compile(pipeline_func,  
  '{}.zip'.format(file_name))

In [85]:
import requests
pipeline_name="qoetest"
pipeline_file = file_name+'.zip'
requests.post("http://tm.traininghost:32002/pipelines/qoetest/upload".format(pipeline_name), files={'file':open(pipeline_file,'rb')})

<Response [200]>